# インポート

In [1]:
import pandas as pd
from pathlib import Path

COMMON_DATA_DIR = Path("..","..","common","data")
RAWDF_DIR = COMMON_DATA_DIR/"rawdf"

##　データ加工

##　レース結果テーブルのデータ加工

In [2]:
df = pd.read_csv(RAWDF_DIR / "results.csv", sep="\t")

In [3]:
# ilocで行列の数を指定
df.iloc[:,0:10]

,race_id,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差
0,202301010101,1,5,5,サトミノキラリ,牡2,55.0,横山武史,1:09.5,NaN
1,202301010101,2,8,8,ベアゴーゴー,牝2,55.0,浜中俊,1:09.5,クビ
2,202301010101,3,6,6,ハピアーザンエバー,牡2,55.0,藤岡佑介,1:10.0,2.1/2
3,202301010101,4,4,4,デビルシズカチャン,牝2,55.0,ルメール,1:10.2,1.1/2
4,202301010101,5,1,1,ウィスピースノー,牝2,55.0,吉田隼人,1:10.3,1/2
...,...,...,...,...,...,...,...,...,...,...
47667,202310030812,8,1,1,リヴァーレ,牝3,53.0,太宰啓介,2:42.9,1.1/4
47668,202310030812,9,6,7,シュアーウィナー,牡5,55.0,川端海翼,2:43.5,3.1/2
47669,202310030812,10,2,2,アウローラシエル,牡3,55.0,和田竜二,2:43.6,3/4
47670,202310030812,11,4,4,ハイグッドエース,牡4,55.0,西塚洸二,2:43.7,3/4


In [4]:
#以下、各列において数値型になっている確認。そうでなければ加工していく
df["着順"]

0         1
1         2
2         3
3         4
4         5
         ..
47667     8
47668     9
47669    10
47670    11
47671    12
Name: 着順, Length: 47672, dtype: object

In [13]:
# dtyopeがobjectになっている。機械学習に通すには数字の型にする必要がある。
# 普通にやるとnumeric関数によりエラーを欠損値として処理するように指定
df["rank"] = pd.to_numeric(df["着順"], errors="coerce")
df["rank"].value_counts(dropna=False)

rank
3.0     3460
1.0     3459
2.0     3456
4.0     3456
5.0     3456
6.0     3443
7.0     3424
8.0     3367
9.0     3259
10.0    3101
11.0    2895
12.0    2637
13.0    2342
14.0    2047
15.0    1708
16.0    1279
NaN      399
17.0     280
18.0     204
Name: count, dtype: int64

In [15]:
df.dropna(subset=["rank"],inplace=True)

In [16]:
df["rank"].value_counts(dropna=False)

rank
3.0     3460
1.0     3459
2.0     3456
4.0     3456
5.0     3456
6.0     3443
7.0     3424
8.0     3367
9.0     3259
10.0    3101
11.0    2895
12.0    2637
13.0    2342
14.0    2047
15.0    1708
16.0    1279
17.0     280
18.0     204
Name: count, dtype: int64

In [17]:
df["馬番"].astype(int)

0        5
1        8
2        6
3        4
4        1
        ..
47667    1
47668    7
47669    2
47670    4
47671    5
Name: 馬番, Length: 47273, dtype: int64

In [19]:
df["性齢"].str[0].value_counts()

性齢
牡    25038
牝    19922
セ     2313
Name: count, dtype: int64

In [24]:
#ラベルエンコーディング：カテゴリ変数を整数にマッピングするデータ加工手法
sex_mapping = {"牡":0, "牝":1, "セ":2}

In [26]:
df["性齢"].str[0].map(sex_mapping).value_counts()

性齢
0    25038
1    19922
2     2313
Name: count, dtype: int64

In [27]:
df["性齢"].str[1:].astype(int)

0        2
1        2
2        2
3        2
4        2
        ..
47667    3
47668    5
47669    3
47670    4
47671    3
Name: 性齢, Length: 47273, dtype: int64

In [28]:
df["斤量"]

0        55.0
1        55.0
2        55.0
3        55.0
4        55.0
         ... 
47667    53.0
47668    55.0
47669    55.0
47670    55.0
47671    52.0
Name: 斤量, Length: 47273, dtype: float64

In [29]:
# １０列目意向を確認
df.iloc[:,10:20]

,単勝,人気,馬体重,調教師,horse_id,jockey_id,trainer_id,owner_id,rank
0,1.2,1.0,452(-4),[東] 鈴木伸尋,2021101429,1170,1026,1026,1.0
1,4.1,2.0,454(+2),[東] 杉浦宏昭,2021105872,1115,1008,1008,2.0
2,59.9,6.0,438(-6),[西] 羽月友彦,2021106854,1093,1091,1091,3.0
3,16.6,3.0,450(+2),[西] 武幸四郎,2021105553,5339,1160,1160,4.0
4,23.9,5.0,434(-10),[西] 今野貞一,2021100648,1095,1128,1128,5.0
...,...,...,...,...,...,...,...,...,...
47667,62.8,11.0,394(+2),[西] 松永幹夫,2020105644,1037,1092,1092,8.0
47668,125.6,12.0,470(-4),[西] 安田隆行,2018106584,1195,438,438,9.0
47669,9.8,5.0,496(+8),[西] 森田直行,2020101781,1018,1142,1142,10.0
47670,37.2,10.0,440(-2),[西] 大根田裕,2019106647,1200,1032,1032,11.0


In [30]:
df["単勝"]

0          1.2
1          4.1
2         59.9
3         16.6
4         23.9
         ...  
47667     62.8
47668    125.6
47669      9.8
47670     37.2
47671      6.2
Name: 単勝, Length: 47273, dtype: object

In [32]:
df["単勝"].astype(float)

0          1.2
1          4.1
2         59.9
3         16.6
4         23.9
         ...  
47667     62.8
47668    125.6
47669      9.8
47670     37.2
47671      6.2
Name: 単勝, Length: 47273, dtype: float64

In [33]:
df["人気"].astype(int)

0         1
1         2
2         6
3         3
4         5
         ..
47667    11
47668    12
47669     5
47670    10
47671     4
Name: 人気, Length: 47273, dtype: int64

In [37]:
df["weight"] = df["馬体重"].str.extract(r"(\d+)").astype(int)
df['weight']

0        452
1        454
2        438
3        450
4        434
        ... 
47667    394
47668    470
47669    496
47670    440
47671    442
Name: weight, Length: 47273, dtype: int64

In [40]:
# 馬体重増減の部分も抜き出す
df["馬体重"].str.extract(r"\((.+)\)").astype(int)
df["weight_diff"] = df["馬体重"].str.extract(r"\((.+)\)").astype(int)
df["weight_diff"]

0        -4
1         2
2        -6
3         2
4       -10
         ..
47667     2
47668    -4
47669     8
47670    -2
47671     6
Name: weight_diff, Length: 47273, dtype: int64

In [5]:
df["枠番"]

0        5
1        8
2        6
3        4
4        1
        ..
47667    1
47668    6
47669    2
47670    4
47671    5
Name: 枠番, Length: 47672, dtype: int64

In [7]:
# 馬テーブルのデータ抽出
df = pd.read_csv(RAWDF_DIR/"horse_results.csv", sep="\t")

In [ ]:
df.iloc[:,10:20]

,オッズ,人気,着順,騎手,斤量,距離,馬場,馬場指数,タイム,着差
0,3.8,2.0,9,津村明秀,57.0,芝1400,良,**,1:21.8,0.2
1,1.9,1.0,1,津村明秀,56.0,芝1400,良,**,1:19.7,-0.5
2,3.0,2.0,2,津村明秀,55.0,芝1400,良,**,1:22.0,0.4
3,11.3,5.0,2,津村明秀,54.0,芝1400,良,**,1:20.3,0.0
4,75.3,11.0,13,内田博幸,57.0,芝1600,稍,**,1:35.3,0.9
...,...,...,...,...,...,...,...,...,...,...
212935,1.3,1.0,1,藤原幹生,56.0,ダ1400,良,**,1:33.1,-0.5
212936,8.2,4.0,3,藤原幹生,56.0,ダ1400,良,**,1:30.8,0.4
212937,9.8,4.0,5,藤原幹生,56.0,ダ800,重,**,0:51.1,2.3
212938,47.5,13.0,14,松若風馬,56.0,ダ1700,良,**,1:49.4,3.4


In [41]:
df.iloc[0:200,0:10]

,horse_id,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番
0,2021101429,2024/11/09,5東京3,晴,10.0,奥多摩S(3勝クラス),NaN,14,5.0,8
1,2021101429,2024/10/06,4東京2,曇,8.0,3歳以上2勝クラス,NaN,13,6.0,8
2,2021101429,2024/09/01,3新潟8,晴,9.0,飯豊特別(2勝クラス),NaN,7,7.0,7
3,2021101429,2024/08/10,3新潟1,晴,10.0,新発田城特別(2勝クラス),NaN,14,8.0,13
4,2021101429,2024/04/06,3中山5,曇,11.0,ニュージーランドT(GII),NaN,16,8.0,16
...,...,...,...,...,...,...,...,...,...,...
195,2020100551,2024/04/19,浦和,晴,9.0,青葉特別(C1),NaN,12,2.0,2
196,2020100551,2024/03/26,浦和,雨,10.0,爽春特別(C2),NaN,12,6.0,8
197,2020100551,2024/02/19,浦和,曇,2.0,ランチタイムチャレン(C2),NaN,12,6.0,8
198,2020100551,2024/01/10,浦和,晴,8.0,C2五 六,NaN,12,8.0,11


In [11]:
df["日付"]

0         2024/11/09
1         2024/10/06
2         2024/09/01
3         2024/08/10
4         2024/04/06
             ...    
212935    2023/12/27
212936    2023/12/06
212937    2023/11/07
212938    2023/09/03
212939    2023/09/03
Name: 日付, Length: 212940, dtype: object

In [12]:
pd.to_datetime(df["日付"])

0        2024-11-09
1        2024-10-06
2        2024-09-01
3        2024-08-10
4        2024-04-06
            ...    
212935   2023-12-27
212936   2023-12-06
212937   2023-11-07
212938   2023-09-03
212939   2023-09-03
Name: 日付, Length: 212940, dtype: datetime64[ns]

In [14]:
df["天気"].value_counts()

天気
晴     126248
曇      64863
雨      13240
小雨      7747
雪        176
小雪       174
Name: count, dtype: int64

In [16]:
df["距離"].str[0].value_counts()


距離
ダ    137369
芝     71285
障      4286
Name: count, dtype: int64

In [18]:
df["距離"].str.extract(r"(\d+)").value_counts()

0   
1400    46707
1200    33221
1800    33097
1600    23996
2000    15773
        ...  
3100        5
2040        1
1990        1
2050        1
1660        1
Name: count, Length: 63, dtype: int64

In [23]:
df["馬場"].value_counts()

馬場
良    135778
稍     37716
重     23705
不     15467
Name: count, dtype: int64

In [24]:
 df["レース名"]

0            奥多摩S(3勝クラス)
1              3歳以上2勝クラス
2            飯豊特別(2勝クラス)
3          新発田城特別(2勝クラス)
4         ニュージーランドT(GII)
               ...      
212935              C21組
212936          マックル賞(C)
212937              C24組
212938             3歳未勝利
212939             3歳未勝利
Name: レース名, Length: 212940, dtype: object

In [51]:
import json
COMMON_DATA_DIR = Path("..","..","common","data")
MAPPING_DIR = COMMON_DATA_DIR/"mapping"
with open(MAPPING_DIR/"race_class.json", "r") as f:
    race_class_mapping = json.load(f)

regex_race_class = "|".join(race_class_mapping)

In [52]:
df["レース名"].str.extract(rf"({regex_race_class})")[0].map(race_class_mapping).value_counts()

0
1.0     44658
2.0     38337
9.0     23789
3.0     10737
0.0     10546
4.0     10152
13.0     5135
6.0      3931
7.0      2177
8.0      1782
5.0        30
Name: count, dtype: int64

In [34]:
df["レース名"].str.extract(rf"({regex_race_class})")[0]

0         3勝クラス
1         2勝クラス
2            特別
3            特別
4           NaN
          ...  
212935      NaN
212936      NaN
212937      NaN
212938      未勝利
212939      未勝利
Name: 0, Length: 212940, dtype: object

In [35]:
regex_race_class

'新馬|未勝利|1勝クラス|2勝クラス|3勝クラス|オープン|G3|G2|G1|特別|500万以下|1000万以下|1600万以下|OP'

In [48]:
filter_df = df[df["horse_id"] == 2017102170]
filter_df["レース名"]

147739           有馬記念(GI)
147740         京都大賞典(GII)
147741           宝塚記念(GI)
147742         天皇賞(春)(GI)
147743         阪神大賞典(GII)
147744           有馬記念(GI)
147745          ジャパンC(GI)
147746         京都大賞典(GII)
147747           宝塚記念(GI)
147748         天皇賞(春)(GI)
147749         阪神大賞典(GII)
147750           有馬記念(GI)
147751           凱旋門賞(GI)
147752           宝塚記念(GI)
147753         天皇賞(春)(GI)
147754         阪神大賞典(GII)
147755           有馬記念(GI)
147756           凱旋門賞(GI)
147757          フォワ賞(GII)
147758         天皇賞(春)(GI)
147759         阪神大賞典(GII)
147760    日刊スポ賞中山金杯(GIII)
147761            菊花賞(GI)
147762         神戸新聞杯(GII)
147763           東京優駿(GI)
147764         京都新聞杯(GII)
147765            皐月賞(GI)
147766       アザレア賞(1勝クラス)
147767       福寿草特別(1勝クラス)
147768              2歳未勝利
147769               2歳新馬
Name: レース名, dtype: object

In [3]:
df = pd.read_csv(RAWDF_DIR/"horse_results.csv", sep="\t")
df

NameError: name 'RAWDF_DIR' is not defined

# 特徴量作成

In [2]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("..","data")

In [58]:
results = pd.read_csv(DATA_DIR/ "01_preprocessed"/ "results.csv", sep="\t")
horse_results = pd.read_csv(DATA_DIR/ "01_preprocessed"/ "horse_results.csv", sep="\t")

In [59]:
results

,race_id,horse_id,jockey_id,trainer_id,owner_id,rank,umaban,wakuban,tansyo_odds,popularity,kinryou,sex,age,weight,weight_diff
0,202301010101,2021100648,1095,1128,1128,5,1,1,23.9,5,55.0,1,2,434,-10
1,202301010101,2021100159,1157,1186,1186,6,2,2,61.8,7,55.0,0,2,454,-6
2,202301010101,2021100265,1197,1192,1192,7,3,3,18.8,4,53.0,1,2,404,-2
3,202301010101,2021105553,5339,1160,1160,4,4,4,16.6,3,55.0,1,2,450,2
4,202301010101,2021101429,1170,1026,1026,1,5,5,1.2,1,55.0,0,2,452,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47268,202310030812,2019102542,1208,1161,1161,1,8,6,3.5,1,53.0,1,4,442,-2
47269,202310030812,2020103754,1116,1104,1104,6,9,7,15.2,8,55.0,0,3,468,0
47270,202310030812,2020100400,1193,1092,1092,4,10,7,23.1,9,50.0,1,3,458,6
47271,202310030812,2020103663,732,1092,1092,2,11,8,5.0,3,55.0,2,3,486,-6


In [60]:
horse_id_list = [2019105346, 2018105165]

In [66]:
horse_results.query("horse_id in @horse_id_list")

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_condition,race_class,n_horses
23408,2018105165,2024-12-22,2.0,20097.2,0.0,0.0,1,2500,0.0,8.0,15
23409,2018105165,2024-11-03,3.0,0.0,NaN,NaN,1,2400,0.0,8.0,13
23410,2018105165,2024-08-18,5.0,700.0,0.6,0.0,1,2000,0.0,7.0,11
23411,2018105165,2024-03-30,2.0,0.0,NaN,NaN,1,2410,0.0,8.0,12
23412,2018105165,2023-12-24,5.0,5000.0,0.3,0.0,1,2500,0.0,8.0,16
23413,2018105165,2023-11-04,3.0,0.0,NaN,0.0,1,2400,0.0,8.0,11
23414,2018105165,2023-08-20,11.0,0.0,3.3,0.0,1,2000,1.0,7.0,15
23415,2018105165,2023-03-25,5.0,0.0,NaN,0.0,1,2410,0.0,8.0,10
23416,2018105165,2022-11-27,2.0,16111.6,0.1,0.0,1,2400,0.0,8.0,18
23417,2018105165,2022-10-30,5.0,2000.0,0.6,0.0,1,2000,0.0,8.0,15


## リスト型になっている特徴量を分類して、新たな特徴量テーブルを作成する

In [114]:
RAWDF_DIR =Path("..","..","common","data","rawdf")
df = pd.read_csv(RAWDF_DIR / "race_info.csv", sep="\t")
df

,race_id,title,info_1,info_2
0,202301010101,2歳未勝利,"['芝右1200m', '天候:晴', '芝:良', '発走:09:50']","['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '..."
1,202301010102,3歳未勝利,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']"
2,202301010103,3歳未勝利,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']"
3,202301010104,3歳未勝利,"['芝右1500m', '天候:晴', '芝:良', '発走:11:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '牝', '指', '..."
4,202301010105,2歳新馬,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']","['2023年7月22日', '1回札幌1日目', '2歳新馬', '混', '指', '馬齢']"
...,...,...,...,...
3451,202310030808,3歳以上1勝クラス,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:13:55']","['2023年9月3日', '3回小倉8日目', '3歳以上1勝クラス', '指', '定量']"
3452,202310030809,西日本新聞杯(2勝),"['芝右1200m', '天候:晴', '芝:良', '発走:14:25', '過去の西日本...","['2023年9月3日', '3回小倉8日目', '3歳以上2勝クラス', '混', '指'..."
3453,202310030810,宮崎ステークス(3勝),"['ダ右1700m', '天候:晴', 'ダート:良', '発走:15:01', '過去の宮...","['2023年9月3日', '3回小倉8日目', '3歳以上3勝クラス', '混', '特指..."
3454,202310030811,第43回小倉2歳ステークス(GIII),"['芝右1200m', '天候:晴', '芝:良', '発走:15:35', '過去の小倉2...","['2023年9月3日', '3回小倉8日目', '2歳オープン', '国際', '特指',..."


## info_1,2の中身が複雑だったので一度strに変換、綺麗なリスト型にして表整理を行う

In [115]:

# print(df.apply(type).unique()) # それぞれの列の型を確認
df["info_1"]


0                  ['芝右1200m', '天候:晴', '芝:良', '発走:09:50']
1                ['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']
2                ['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']
3                  ['芝右1500m', '天候:晴', '芝:良', '発走:11:25']
4                ['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']
                              ...                        
3451             ['ダ右1000m', '天候:晴', 'ダート:良', '発走:13:55']
3452    ['芝右1200m', '天候:晴', '芝:良', '発走:14:25', '過去の西日本...
3453    ['ダ右1700m', '天候:晴', 'ダート:良', '発走:15:01', '過去の宮...
3454    ['芝右1200m', '天候:晴', '芝:良', '発走:15:35', '過去の小倉2...
3455               ['芝右2600m', '天候:晴', '芝:良', '発走:16:15']
Name: info_1, Length: 3456, dtype: object

In [90]:
# info_1列の型を文字列に変換し、[]などを削除して、リスト型に変換する。
df["info_1"] = df["info_1"].astype(str)
df["info_1"] = df["info_1"].str.strip("[]").str.replace("'", "").str.split(", ")
# print(df['info_1'].apply(type).unique())
df["info_1"]


0                     [芝右1200m, 天候:晴, 芝:良, 発走:09:50]
1                   [ダ右1000m, 天候:晴, ダート:良, 発走:10:25]
2                   [ダ右1700m, 天候:晴, ダート:良, 発走:10:55]
3                     [芝右1500m, 天候:晴, 芝:良, 発走:11:25]
4                   [ダ右1700m, 天候:晴, ダート:良, 発走:12:15]
                            ...                     
3451                [ダ右1000m, 天候:晴, ダート:良, 発走:13:55]
3452       [芝右1200m, 天候:晴, 芝:良, 発走:14:25, 過去の西日本新聞杯]
3453    [ダ右1700m, 天候:晴, ダート:良, 発走:15:01, 過去の宮崎ステークス]
3454    [芝右1200m, 天候:晴, 芝:良, 発走:15:35, 過去の小倉2歳ステークス]
3455                  [芝右2600m, 天候:晴, 芝:良, 発走:16:15]
Name: info_1, Length: 3456, dtype: object

In [93]:
df["info_2"]

0       ['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '...
1           ['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']
2           ['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']
3       ['2023年7月22日', '1回札幌1日目', '3歳未勝利', '牝', '指', '...
4       ['2023年7月22日', '1回札幌1日目', '2歳新馬', '混', '指', '馬齢']
                              ...                        
3451     ['2023年9月3日', '3回小倉8日目', '3歳以上1勝クラス', '指', '定量']
3452    ['2023年9月3日', '3回小倉8日目', '3歳以上2勝クラス', '混', '指'...
3453    ['2023年9月3日', '3回小倉8日目', '3歳以上3勝クラス', '混', '特指...
3454    ['2023年9月3日', '3回小倉8日目', '2歳オープン', '国際', '特指',...
3455    ['2023年9月3日', '3回小倉8日目', '3歳以上1勝クラス', '混', '特指...
Name: info_2, Length: 3456, dtype: object

In [106]:
df_expanded = pd.DataFrame(df["info_1"].to_list(), columns=["コース情報", "天候", "状態", "発走時刻","発送時間", "発送時間1"])
# df_expanded["コース"] = df_expanded["コース情報"].str.split(r"右|左").str[0]
df_expanded["方向"] = df_expanded["コース情報"].str.extract(r"(右|左)")  # '右' または '左' を抽出


,コース情報,天候,状態,発走時刻,発送時間,発送時間1,方向
0,芝右1200m,天候:晴,芝:良,発走:09:50,None,None,右
1,ダ右1000m,天候:晴,ダート:良,発走:10:25,None,None,右
2,ダ右1700m,天候:晴,ダート:良,発走:10:55,None,None,右
3,芝右1500m,天候:晴,芝:良,発走:11:25,None,None,右
4,ダ右1700m,天候:晴,ダート:良,発走:12:15,None,None,右
...,...,...,...,...,...,...,...
3451,ダ右1000m,天候:晴,ダート:良,発走:13:55,None,None,右
3452,芝右1200m,天候:晴,芝:良,発走:14:25,過去の西日本新聞杯,None,右
3453,ダ右1700m,天候:晴,ダート:良,発走:15:01,過去の宮崎ステークス,None,右
3454,芝右1200m,天候:晴,芝:良,発走:15:35,過去の小倉2歳ステークス,None,右


In [110]:
# info_1列を展開して、新しいDataFrameを作成する.リストの要素に対してコラム数が少ないとエラーが出るので、コラム数を無理やり５個にする
df_expanded = pd.DataFrame(df["info_1"].to_list(), columns=["コース情報", "天候", "状態", "発走時刻","発送時間", "発送時間1"])
df_expanded["コース"] = df_expanded["コース情報"].str.split(r"右|左").str[0]  # コース (例: 'ダ')
df_expanded["方向"] = df_expanded["コース情報"].str.extract(r"(右|左)")  # '右' または '左' を抽出
df_expanded["距離"] = df_expanded["コース情報"].str.extract(r'(\d+)')    # 距離 (例: '1000')
df_expanded["天候"] = df_expanded["天候"].str.split(":").str[1]         # 天候 (例: '晴')
df_expanded["状態"] = df_expanded["状態"].str.split(":").str[1]         # 状態 (例: '良')
df_expanded["発走時刻"] = df_expanded["発走時刻"].str.split(":").str[1] # 発走時刻 (例: '10:25')
df_expanded1 = df_expanded.drop(columns=["コース情報", "発走時刻", "発送時間","発送時間1"])
# print(df_expanded.apply(type).unique())
df_expanded1



,天候,状態,コース,方向,距離
0,晴,良,芝,右,1200
1,晴,良,ダ,右,1000
2,晴,良,ダ,右,1700
3,晴,良,芝,右,1500
4,晴,良,ダ,右,1700
...,...,...,...,...,...
3451,晴,良,ダ,右,1000
3452,晴,良,芝,右,1200
3453,晴,良,ダ,右,1700
3454,晴,良,芝,右,1200


In [94]:
# info_2についても同様の操作を行う
# info_2列の型を文字列に変換し、[]などを削除して、リスト型に変換する。
df["info_2"] = df["info_2"].astype(str)
df["info_2"] = df["info_2"].str.strip("[]").str.replace("'", "").str.split(", ")
# print(df['info_1'].apply(type).unique())
df["info_2"]



0           [2023年7月22日, 1回札幌1日目, 2歳未勝利, 混, 指, 馬齢]
1              [2023年7月22日, 1回札幌1日目, 3歳未勝利, 指, 馬齢]
2              [2023年7月22日, 1回札幌1日目, 3歳未勝利, 指, 馬齢]
3           [2023年7月22日, 1回札幌1日目, 3歳未勝利, 牝, 指, 馬齢]
4            [2023年7月22日, 1回札幌1日目, 2歳新馬, 混, 指, 馬齢]
                           ...                    
3451        [2023年9月3日, 3回小倉8日目, 3歳以上1勝クラス, 指, 定量]
3452     [2023年9月3日, 3回小倉8日目, 3歳以上2勝クラス, 混, 指, 定量]
3453    [2023年9月3日, 3回小倉8日目, 3歳以上3勝クラス, 混, 特指, 定量]
3454      [2023年9月3日, 3回小倉8日目, 2歳オープン, 国際, 特指, 馬齢]
3455    [2023年9月3日, 3回小倉8日目, 3歳以上1勝クラス, 混, 特指, 定量]
Name: info_2, Length: 3456, dtype: object

In [74]:
df = pd.DataFrame(df["info_2"].to_list(), columns=["日付", "レース情報", "レース名", "属性1", "属性2", "条件1","条件2","条件3"])

# 上から3つ目の要素を取り出す
df_subset = df.iloc[:, :3]
df_subset


,日付,レース情報,レース名
0,2023年7月22日,1回札幌1日目,2歳未勝利
1,2023年7月22日,1回札幌1日目,3歳未勝利
2,2023年7月22日,1回札幌1日目,3歳未勝利
3,2023年7月22日,1回札幌1日目,3歳未勝利
4,2023年7月22日,1回札幌1日目,2歳新馬
...,...,...,...
3451,2023年9月3日,3回小倉8日目,3歳以上1勝クラス
3452,2023年9月3日,3回小倉8日目,3歳以上2勝クラス
3453,2023年9月3日,3回小倉8日目,3歳以上3勝クラス
3454,2023年9月3日,3回小倉8日目,2歳オープン


In [ ]:
# df_subset["レース会場"] = df_subset["レース情報"].str.extract(r"(\D+)")  # 数字以外の文字列を抽出
# df_subset["レース会場"] = df_subset["レース会場"].str.replace("回", "", regex=False)  # "回"を削除
df_subset.loc[:,"レース会場"] = df_subset["レース情報"].str.extract(r"(\D+)")[0].str.replace("回", "", regex=False)
df_expanded2 = df_subset.drop(columns=["レース情報"])

In [111]:
# データフレームをつなげてinfo_1, info_2を削除する
df_combined = pd.concat([df, df_expanded1, df_expanded2], axis=1)
df_combined = df_combined.drop(columns=["info_1","info_2","title"])
df_combined


,race_id,天候,状態,コース,方向,距離,日付,レース名,レース会場
0,202301010101,晴,良,芝,右,1200,2023年7月22日,2歳未勝利,札幌
1,202301010102,晴,良,ダ,右,1000,2023年7月22日,3歳未勝利,札幌
2,202301010103,晴,良,ダ,右,1700,2023年7月22日,3歳未勝利,札幌
3,202301010104,晴,良,芝,右,1500,2023年7月22日,3歳未勝利,札幌
4,202301010105,晴,良,ダ,右,1700,2023年7月22日,2歳新馬,札幌
...,...,...,...,...,...,...,...,...,...
3451,202310030808,晴,良,ダ,右,1000,2023年9月3日,3歳以上1勝クラス,小倉
3452,202310030809,晴,良,芝,右,1200,2023年9月3日,3歳以上2勝クラス,小倉
3453,202310030810,晴,良,ダ,右,1700,2023年9月3日,3歳以上3勝クラス,小倉
3454,202310030811,晴,良,芝,右,1200,2023年9月3日,2歳オープン,小倉


##  info_1の中身がをリスト型として処理できると思っていたので、そのまま展開していたが、リスト型になっていないことがわかったので、事故ってる

In [6]:
condi_cols = ["距離・方向", "天候","馬場", "発送時間"]
state_cols = ["開催日","場所","クラス", '条件1', '条件2', '条件3']

In [10]:
classified = []
for row in df["info_1"]:
        # 各要素を抽出
        course = row[0]  # コース情報 (例: 芝右1200m)
        weather = row[1].split(':')[1]  # 天候 (例: 晴)
        surface_condition = row[2].split(':')[1]  # 芝/ダート状態 (例: 良)
        
        classified.append({
            'コース情報': course,
            '天候': weather,
            '状態': surface_condition,
        })

IndexError: list index out of range

In [34]:
data = df["info_1"]
# df = pd.DataFrame(data, columns=['距離・方向', '天候', '馬場状態', '発走時間'])
# df

KeyError: 'info_1'

In [12]:
df[condi_cols] = pd.DataFrame(df['info_1'].tolist(), index=df.index)
df[state_cols] = pd.DataFrame(df['info_2'].tolist(), index=df.index)

ValueError: Columns must be same length as key

In [15]:
import numpy as np

# 最大長に合わせてリストを補完
max_length = df['info_1'].apply(len).max()
df['info_1'] = df['info_1'].apply(lambda x: x + [np.nan] * (max_length - len(x)))

TypeError: can only concatenate str (not "list") to str

In [22]:
# info_1が辞書型だとnumpyを使えないのですべての要素をリストに変換
df['info_1'] = df['info_1'].apply(lambda x: x if isinstance(x, list) else [x])
# print(df['info_1'].apply(type).unique())
df["info_1"]

0             [['芝右1200m', '天候:晴', '芝:良', '発走:09:50']]
1           [['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']]
2           [['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']]
3             [['芝右1500m', '天候:晴', '芝:良', '発走:11:25']]
4           [['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']]
5             [['芝右1800m', '天候:晴', '芝:良', '発走:12:45']]
6             [['芝右1200m', '天候:晴', '芝:良', '発走:13:15']]
7           [['ダ右1700m', '天候:晴', 'ダート:良', '発走:13:45']]
8    [['芝右1800m', '天候:晴', '芝:良', '発走:14:15', '過去の北辰...
9    [['芝右2000m', '天候:晴', '芝:良', '発走:14:50', '過去のライ...
Name: info_1, dtype: object

In [27]:
import numpy as np

# 最大長に合わせてリストを補完
max_length = df['info_1'].apply(len).max()
df['info_1'] = df['info_1'].apply(lambda x: x + [np.nan] * (max_length - len(x)))

np.int64(1)

In [24]:
df["info_1"]

0             [['芝右1200m', '天候:晴', '芝:良', '発走:09:50']]
1           [['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']]
2           [['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']]
3             [['芝右1500m', '天候:晴', '芝:良', '発走:11:25']]
4           [['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']]
5             [['芝右1800m', '天候:晴', '芝:良', '発走:12:45']]
6             [['芝右1200m', '天候:晴', '芝:良', '発走:13:15']]
7           [['ダ右1700m', '天候:晴', 'ダート:良', '発走:13:45']]
8    [['芝右1800m', '天候:晴', '芝:良', '発走:14:15', '過去の北辰...
9    [['芝右2000m', '天候:晴', '芝:良', '発走:14:50', '過去のライ...
Name: info_1, dtype: object

In [25]:
df[condi_cols] = pd.DataFrame(df['info_1'].tolist(), index=df.index)

ValueError: Columns must be same length as key

In [30]:
df['info_1'] = df['info_1'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else [])
# print(df['info_1'].apply(type).unique())
df["info_1"]

0    []
1    []
2    []
3    []
4    []
5    []
6    []
7    []
8    []
9    []
Name: info_1, dtype: object

In [112]:
# 各列の型を確認

df_combined

,race_id,天候,状態,コース,方向,距離,日付,レース名,レース会場
0,202301010101,晴,良,芝,右,1200,2023年7月22日,2歳未勝利,札幌
1,202301010102,晴,良,ダ,右,1000,2023年7月22日,3歳未勝利,札幌
2,202301010103,晴,良,ダ,右,1700,2023年7月22日,3歳未勝利,札幌
3,202301010104,晴,良,芝,右,1500,2023年7月22日,3歳未勝利,札幌
4,202301010105,晴,良,ダ,右,1700,2023年7月22日,2歳新馬,札幌
...,...,...,...,...,...,...,...,...,...
3451,202310030808,晴,良,ダ,右,1000,2023年9月3日,3歳以上1勝クラス,小倉
3452,202310030809,晴,良,芝,右,1200,2023年9月3日,3歳以上2勝クラス,小倉
3453,202310030810,晴,良,ダ,右,1700,2023年9月3日,3歳以上3勝クラス,小倉
3454,202310030811,晴,良,芝,右,1200,2023年9月3日,2歳オープン,小倉


In [120]:
 # evalで文字列型の列をリスト型に変換し、一時的な列を作成
df["tmp"] = df["info_1"].map(lambda x: eval(x)[1])
df

,race_id,title,info_1,info_2,tmp
0,202301010101,2歳未勝利,"['芝右1200m', '天候:晴', '芝:良', '発走:09:50']","['2023年7月22日', '1回札幌1日目', '2歳未勝利', '混', '指', '...",天候:晴
1,202301010102,3歳未勝利,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:10:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']",天候:晴
2,202301010103,3歳未勝利,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:10:55']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '指', '馬齢']",天候:晴
3,202301010104,3歳未勝利,"['芝右1500m', '天候:晴', '芝:良', '発走:11:25']","['2023年7月22日', '1回札幌1日目', '3歳未勝利', '牝', '指', '...",天候:晴
4,202301010105,2歳新馬,"['ダ右1700m', '天候:晴', 'ダート:良', '発走:12:15']","['2023年7月22日', '1回札幌1日目', '2歳新馬', '混', '指', '馬齢']",天候:晴
...,...,...,...,...,...
3451,202310030808,3歳以上1勝クラス,"['ダ右1000m', '天候:晴', 'ダート:良', '発走:13:55']","['2023年9月3日', '3回小倉8日目', '3歳以上1勝クラス', '指', '定量']",天候:晴
3452,202310030809,西日本新聞杯(2勝),"['芝右1200m', '天候:晴', '芝:良', '発走:14:25', '過去の西日本...","['2023年9月3日', '3回小倉8日目', '3歳以上2勝クラス', '混', '指'...",天候:晴
3453,202310030810,宮崎ステークス(3勝),"['ダ右1700m', '天候:晴', 'ダート:良', '発走:15:01', '過去の宮...","['2023年9月3日', '3回小倉8日目', '3歳以上3勝クラス', '混', '特指...",天候:晴
3454,202310030811,第43回小倉2歳ステークス(GIII),"['芝右1200m', '天候:晴', '芝:良', '発走:15:35', '過去の小倉2...","['2023年9月3日', '3回小倉8日目', '2歳オープン', '国際', '特指',...",天候:晴


In [125]:
# df["info_1"].str.extract(r"天候:(\w+)")[0]
pd.to_datetime(df["info_2"].map(lambda x: eval(x)[0]), format="%Y年%m月%d日")

0      2023-07-22
1      2023-07-22
2      2023-07-22
3      2023-07-22
4      2023-07-22
          ...    
3451   2023-09-03
3452   2023-09-03
3453   2023-09-03
3454   2023-09-03
3455   2023-09-03
Name: info_2, Length: 3456, dtype: datetime64[ns]

#　払い戻しテーブルを加工

In [1]:
import json
import pandas as pd
from pathlib import Path

COMMON_DATA_DIR = Path("..","..","common","data")
RAWDF_DIR = COMMON_DATA_DIR/"rawdf"

In [4]:
raw = pd.read_csv(RAWDF_DIR/"return_tables.csv",sep="\t", index_col=0)
raw

,0,1,2,3
race_id,,,,
202301010101,単勝,5,120,1
202301010101,複勝,5 8 6,100 110 310,1 2 6
202301010101,馬連,5 - 8,170,1
202301010101,ワイド,5 - 8 5 - 6 6 - 8,120 600 880,1 8 12
202301010101,馬単,5 → 8,210,1
...,...,...,...,...
202310030812,馬連,8 - 11,940,1
202310030812,ワイド,8 - 11 6 - 8 6 - 11,380 410 470,1 2 4
202310030812,馬単,8 → 11,1580,1


In [6]:
# 表示させたいものを右に行、左に列を指定
df = raw.loc[:,["1","2"]]
df

,1,2
race_id,,
202301010101,5,120
202301010101,5 8 6,100 110 310
202301010101,5 - 8,170
202301010101,5 - 8 5 - 6 6 - 8,120 600 880
202301010101,5 → 8,210
...,...,...
202310030812,8 - 11,940
202310030812,8 - 11 6 - 8 6 - 11,380 410 470
202310030812,8 → 11,1580


In [10]:
df["2"].str.split(" ")

race_id
202301010101           [120, 600, 880]
202301010102       [560, 2,710, 4,990]
202301010103           [370, 510, 360]
202301010104       [1,140, 1,620, 520]
202301010105           [300, 380, 300]
                         ...          
202310030808         [700, 2,800, 910]
202310030809    [4,300, 5,020, 16,830]
202310030810       [450, 1,240, 1,250]
202310030811           [530, 630, 530]
202310030812           [380, 410, 470]
Name: 2, Length: 3456, dtype: object

In [12]:
df["1"].str.replace(" - ","-").str.split(" ")

race_id
202301010101          [5-8, 5-6, 6-8]
202301010102          [8-9, 3-9, 3-8]
202301010103        [4-11, 9-11, 4-9]
202301010104        [1-9, 9-13, 1-13]
202301010105        [4-10, 4-7, 7-10]
                        ...          
202310030808        [4-10, 3-10, 3-4]
202310030809    [14-17, 12-17, 12-14]
202310030810        [5-9, 5-12, 9-12]
202310030811        [9-10, 5-10, 5-9]
202310030812        [8-11, 6-8, 6-11]
Name: 1, Length: 3456, dtype: object

In [ ]:
# regex=Trueを指定しないとデータの中身が" - "だけでないと変換してくれない。指定することでそれを含むものも対称にして変換してくれる
df.replace(" - ","-", regex = True)

,1,2
race_id,,
202301010101,5-8 5-6 6-8,120 600 880
202301010102,8-9 3-9 3-8,"560 2,710 4,990"
202301010103,4-11 9-11 4-9,370 510 360
202301010104,1-9 9-13 1-13,"1,140 1,620 520"
202301010105,4-10 4-7 7-10,300 380 300
...,...,...
202310030808,4-10 3-10 3-4,"700 2,800 910"
202310030809,14-17 12-17 12-14,"4,300 5,020 16,830"
202310030810,5-9 5-12 9-12,"450 1,240 1,250"


## DataFrameで列ごとに処理を適応したい場合：apply(lamdba:x:処理(x))を使う

In [17]:
# lambdaはfor文のような感じ１ループ目で１列目を処理している
# split()はデフォルトで半角スペースになっているので指定しなくてもよい
#explodeで横につながっているものを縦に爆発させる
df.replace(" - ","-", regex=True).apply(lambda x: x.str.split()).explode(["1","2"])

,1,2
race_id,,
202301010101,5-8,120
202301010101,5-6,600
202301010101,6-8,880
202301010102,8-9,560
202301010102,3-9,"2,710"
...,...,...
202310030811,5-10,630
202310030811,5-9,530
202310030812,8-11,380


In [22]:
tmp = (
    df.replace(" - ","-", regex=True)
    .apply(lambda x: x.str.split())
    .explode(["1","2"])
    .apply(lambda x: x.str.split("-"))
    .explode("2")
    .replace(",","", regex=True)
)
tmp["2"] = tmp["2"].astype(int)

In [23]:
tmp

,1,2
race_id,,
202301010101,"[5, 8]",120
202301010101,"[5, 6]",600
202301010101,"[6, 8]",880
202301010102,"[8, 9]",560
202301010102,"[3, 9]",2710
...,...,...
202310030811,"[5, 10]",630
202310030811,"[5, 9]",530
202310030812,"[8, 11]",380


In [24]:
raw.head(10)

,0,1,2,3
race_id,,,,
202301010101,単勝,5,120,1
202301010101,複勝,5 8 6,100 110 310,1 2 6
202301010101,馬連,5 - 8,170,1
202301010101,ワイド,5 - 8 5 - 6 6 - 8,120 600 880,1 8 12
202301010101,馬単,5 → 8,210,1
202301010101,三連複,5 - 6 - 8,1120,4
202301010101,三連単,5 → 8 → 6,2680,9
202301010102,単勝,9,240,1
202301010102,複勝,9 8 3,140 280 990,1 5 11


In [25]:
df = raw.loc[raw["0"]=="馬単",["1","2"]]
df

,1,2
race_id,,
202301010101,5 → 8,210
202301010102,9 → 8,2020
202301010103,11 → 4,2580
202301010104,9 → 1,10010
202301010105,4 → 10,1430
...,...,...
202310030808,10 → 4,3320
202310030809,17 → 14,26080
202310030810,5 → 9,2130


In [31]:
(
    df.replace(" → ","-", regex=True)
    .apply(lambda x: x.str.split())
    .explode(["1","2"])
    .apply(lambda x: x.str.split("-"))
    .explode("2")
    .replace(",","", regex=True)
)

,1,2
race_id,,
202301010101,"[5, 8]",210
202301010102,"[9, 8]",2020
202301010103,"[11, 4]",2580
202301010104,"[9, 1]",10010
202301010105,"[4, 10]",1430
...,...,...
202310030808,"[10, 4]",3320
202310030809,"[17, 14]",26080
202310030810,"[5, 9]",2130


In [34]:
df = raw.loc[raw["0"]=="三連複",["1","2"]]
df

,1,2
race_id,,
202301010101,5 - 6 - 8,1120
202301010102,3 - 8 - 9,18440
202301010103,4 - 9 - 11,1930
202301010104,1 - 9 - 13,7060
202301010105,4 - 7 - 10,1040
...,...,...
202310030808,3 - 4 - 10,8280
202310030809,12 - 14 - 17,130230
202310030810,5 - 9 - 12,4500


In [43]:
(
    df.replace(" - ","-", regex=True)
    .apply(lambda x: x.str.split())
    .explode(["1","2"])
    .apply(lambda x: x.str.split("-"))
    .explode("2")
    .replace(",","", regex=True)
).index.value_counts()

race_id
202305050211    2
202301020112    2
202301010403    2
202305020904    2
202305040805    2
               ..
202301010304    1
202301010305    1
202301010306    1
202301010307    1
202301010308    1
Name: count, Length: 3456, dtype: int64

In [54]:
(
    raw[["0","1","2"]]
    .replace(" (-|→) ","-", regex=True)
    .apply(lambda x: x.str.split())
    .explode(["1","2"])
    .explode("0")
    .apply(lambda x: x.str.split("-"))
    .explode(["0","2"])
    .replace(",","", regex=True)
)

,0,1,2
race_id,,,
202301010101,単勝,[5],120
202301010101,複勝,[5],100
202301010101,複勝,[8],110
202301010101,複勝,[6],310
202301010101,馬連,"[5, 8]",170
...,...,...,...
202310030812,ワイド,"[6, 8]",410
202310030812,ワイド,"[6, 11]",470
202310030812,馬単,"[8, 11]",1580


In [63]:
import preprocessing
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
%autoreload

In [68]:
df = preprocessing.process_return_tables()

In [69]:
df

,bet_type,umaban,return
race_id,,,
202301010101,単勝,[5],120
202301010101,複勝,[5],100
202301010101,複勝,[8],110
202301010101,複勝,[6],310
202301010101,馬連,"[5, 8]",170
...,...,...,...
202310030812,ワイド,"[6, 8]",410
202310030812,ワイド,"[6, 11]",470
202310030812,馬単,"[8, 11]",1580


###　データチェック

In [71]:
tmp = df.loc[df["bet_type"] == "単勝", ["umaban", "return"]]
tmp

,umaban,return
race_id,,
202301010101,[5],120
202301010102,[9],240
202301010103,[11],840
202301010104,[9],1970
202301010105,[4],460
...,...,...
202310030808,[10],830
202310030809,[17],850
202310030810,[5],500


In [74]:
tmp = tmp.explode("umaban").astype(int)

In [75]:
tmp.index.nunique()

3456

In [76]:
tmp.index.value_counts()

race_id
202302010610    2
202306040308    2
202304010810    2
202310030309    1
202310030310    1
               ..
202301010304    1
202301010305    1
202301010306    1
202301010307    1
202301010308    1
Name: count, Length: 3456, dtype: int64

In [ ]:
from train import Trainer
trainer = Trainer()


